# ROOT without ROOT!

In this brief tutorial I'll go through opening a ROOT file, performing some basic cuts, saving our results to a dataframe and making some plots.

This tutorial is on GitHub, and can be found at https://github.com/professor-calculus/AlexDataFramesTutorial.git

![alt text](./img.jpg)

First let's load up our dependencies, we won't need many.
Spoiler: ROOT isnt one of them!

In [ ]:
import math
import pandas as pd
import uproot
from tqdm import tqdm_notebook as tqdm

Now, let's open the ROOT file from Delphes:

In [ ]:
tree = uproot.open('./delphes.root')["Delphes"]

We can also take a look at the branches, and the leaves within each branch etc using keys():

In [ ]:
tree.keys()

In [ ]:
tree['Jet'].keys()

Let's define some placeholder variables we'll use: arrays which will contain per-event information.

In [ ]:
# Cuts: 200GeV MHT, 800GeV HT, >=3 jets, >=1 b-jet
mht_min = 400.
ht_min = 900.
njet_min = 3
nbjet_min = 2

total_n_entries = len(tree['ScalarHT.HT'])
print('Tree of {0} entries read in'.format(total_n_entries))

### Iterating over a ROOT tree:

To iterate over the ROOT tree we use the inventively titles 'iterate()' function in uproot.

In [ ]:
# Initialise the variables we'll write to:
mht = [] # Missing HT: Vector sum of jet pT
ht = [] # Scalar HT: Scalar sum of jet pT
njets = [] # Number of jets which pass certain criteris (ID etc)
nbjets = [] # Number of b-tagged jets...
event_passes_bool = [] # Does event pass cuts?
n_eventpass = 0 # Number of events which passed the cuts

for HT, JetPt, JetEta, JetPhi, JetBtag in tqdm(uproot.iterate('./delphes.root', 'Delphes', ["ScalarHT.HT", "Jet.PT", "Jet.Eta", "Jet.Phi", "Jet.BTag"], outputtype=tuple), desc='Go Go Go!'):
    for HT_i, JetPt_i, JetEta_i, JetPhi_i, JetBtag_i in zip(HT, JetPt, JetEta, JetPhi, JetBtag):
        
        # Reset some variables:
        nJet = 0
        nBJet = 0
        mht_x = 0.
        mht_y = 0.
        
        # Easy one first:
        ht.append(HT_i[0])
        
        # Loop over the jets in the event
        for JetPt_j, JetEta_j, JetPhi_j, JetBtag_j in zip(JetPt_i, JetEta_i, JetPhi_i, JetBtag_i):
            # Only include central jets with decent pT to avoid pileup contributions etc
            if JetPt_j > 40. and abs(JetEta_j) < 2.4:
                nJet += 1
                mht_x += -1. * JetPt_j * math.cos(JetPhi_j)
                mht_y += JetPt_j * math.sin(JetPhi_j)
                
                # Does this jet have a b-tag?
                if JetBtag_j:
                    nBJet += 1
        
        # Missing-HT from its components
        mht_tmp = math.sqrt(mht_x**2 + mht_y**2)
        mht.append(mht_tmp)
        
        njets.append(nJet)
        nbjets.append(nBJet)
        
        # Does this event pass all cuts?
        pass_cuts = True
        if mht_tmp < mht_min: pass_cuts = False
        if HT_i[0] < ht_min: pass_cuts = False
        if nJet < njet_min: pass_cuts = False
        if nBJet < nbjet_min: pass_cuts = False
        event_passes_bool.append(pass_cuts)
        if pass_cuts: n_eventpass += 1
        
percentage = 100.*float(n_eventpass)/float(total_n_entries)
print('{0} of {1} events ({2}%) passed all cuts'.format(n_eventpass, total_n_entries, percentage))

## Writing the variables to a dataframe
df.head() shows first 5 rows (events)

In [ ]:
df = pd.DataFrame({
    'HT': ht,
    'MHT': mht,
    'NJets': njets,
    'NBJets': nbjets,
    'Passes_Cuts': event_passes_bool,
})
df.head()

## Save output
Save to a .txt file to access later

In [ ]:
df.to_csv('DataFrame.txt', sep='\t', index=False)